<a href="https://colab.research.google.com/github/Fahad-Aslam/LangChain/blob/main/Embedding_based_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading the model

In [ ]:
!pip install langchain huggingface_hub > /dev/null
!pip install google-search-resultsfrom langchain import HuggingFaceHub

In [ ]:
pip install huggingface_hub

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
huggingfacehub_api_token="huggingface_api_token"

In [ ]:
# Falcon 7Billion Chat-model


# # Falcon 7Billion Chat-model
# repo_id="TheBloke/Nous-Hermes-13B-GPTQ"
#repo_id="TheBloke/Llama-2-13B-GGML"
repo_id = "tiiuae/falcon-7b-instruct"
# repo_id2 = "google/flan-t5-xxl"
llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token,
                     repo_id=repo_id,
                     model_kwargs={"temperature": 0.5,
                                   "max_new_tokens":200,
                                  #  "min_length": 200,
                                   "max_length": 200,
                                   "num_return_sequences":1
                                  #  "top_k": 50,
                                  #  "top_p": .95,
                                  #  "do_sample": True,
                                  #  "early_stopping": False,
                                  #  "num_beams": 1,
                                  #  "no_repeat_ngram_size": 3
                                   })




#Very simple sequence

In [ ]:
!pip install langchain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = llm

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = llm
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [ ]:
review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...


The play opens at sunset on a beach, where a man and a woman meet by chance. As the sun fades away and the waves crash against the shore, they share a connection that brings them together. However, their moment of happiness is short lived as an unexpected tragedy occurs. As the play unfolds, the couple struggles to come to terms with their loss, ultimately leading to a heartwarming and emotional conclusion.


The emotional resonance of this play is palpable, as it delicately explores the fragility of human connection and the devastating impact of sudden loss. The performances from the two leads are outstanding, and their palpable chemistry is a joy to behold. The playwright has crafted a touching and thought-provoking piece that will leave you feeling moved and inspired.

> Finished chain.


#2

In [ ]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = llm
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [ ]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = llm
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [ ]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': '\n\nIn a world where social class rules, young artist, Mr. William Turner, falls in love with the married and wealthy Mrs. Catherine More. As their passionate affair unfolds, it becomes clear that their love will have tragic consequences. The play explores themes of forbidden love, societal expectations, and the devastating effects of the rigid class structure. With a gripping plot and poignant performances, Tragedy at Sunset on the Beach is a timeless tale of unrequited love that transcends societal barriers.',
 'review': '\n\nTragedy at Sunset on the Beach is a gripping, timely tale of forbidden love that transcends societal barriers. With a captivating plot and stellar performances, the play offers a poignant examination of unrequited love, as well as a haunting exploration of class structure. This timeless classic is a must-see for theater-goers seeking a thought-provoking evening of drama.'}

#Memory in Sequential Chains

In [ ]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = llm
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': "\nAre you ready to experience a timeless classic this holiday season? Join us at the Theater in the Park for #TragedyAtSunset, a gripping tale of unrequited love. Get your tickets now to avoid disappointment. #TragedyAtSunset #TheaterInThePark #HolidayTheater\n\nDon't miss this classic play this December. Get your tickets now to experience a timeless tale that transcends societal barriers. #TragedyAtSunset #TheaterInThePark #HolidayTheater"}

#Transformation

In [ ]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




# documents = TextLoader('/content/drive/MyDrive/finma.txt').load()

with open("/content/drive/MyDrive/finma.txt") as f:
    state_of_the_union = f.read()

print(state_of_the_union)

In [ ]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [ ]:
template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [ ]:
sequential_chain.run(state_of_the_union)

#Router

In [ ]:
from langchain.chains.router import MultiPromptChain
# from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [ ]:
llm = llm

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")

####LLMRouterChain

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
    # handle_parsing_errors=True
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
print(chain.run("What is black body radiation?"))

In [ ]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"
    )
)

In [ ]:
print(chain.run("What is the name of the type of cloud that rins"))

#EmbeddingRouterChain

###loading the embeddings

In [ ]:
!pip install sentence_transformers
!pip install chromadb==0.3.29


In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import VectorDBQA


In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
#embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embeddings = SentenceTransformerEmbeddings(model_name="tiiuae/falcon-7b-instruct")

In [ ]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
# from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
names_and_descriptions = [
    ("physics", ["for questions about physics"]),
    ("math", ["for questions about math"]),
]

In [ ]:
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, embeddings, routing_keys=["input"]
)

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.

A) A type of radiation that can be absorbed by all matter
B) A type of radiation that can be emitted by all matter
C) A type of radiation that can be both absorbed and emitted by all matter
D) A type of radiation that is always absorbed by all matter

Answer: D) A type of radiation that is always absorbed by all matter.


In [ ]:
print(
    chain.run(
        "What is a set"
    )
)



> Entering new MultiPromptChain chain...
math: {'input': 'What is a set'}
> Finished chain.
 of 10 numbers between 1 and 100, where each number is 2 more than the previous number?

You can answer this question by breaking it down into smaller parts. First, you need to find the first number in the set. This is 1. Then, you need to add 2 to get the next number, which is 3. So the next number in the set is 3. Then, you need to add 2 to get the next number, which is 5. So the next number in the set is 5. Then, you need to add 2 to get the next number, which is 7. So the next number in the set is 7. And so on until you reach 100.

Once you have all the numbers, you can add them up to get the total sum of the set. The sum of the numbers in the set is 1000. Therefore, the set of 10 numbers between
